# Building a web app

<html><div style="background-color:#ffeeee; border-radius: 8px; border: solid 1px #aa8888; padding: 0.5em 1em 1em 1em;">
            
<h3>NOTE</h3>

<p>Much of the code in this notebook doesn't work in a Notebook. It needs to reside in a `flask` application. Usually this means putting it in a file called `app.py`, which can be run by typing `flask run` on the command line in the same directory.</p>
    
</div></html>

## 0. Minimal

This is the smallest possible flask application. It serves a single string on the root path.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def root():
    """
    Simplest possible. Return a string.
    """
    return "Hello world!"

❗ **IMPORTANT**

To run, we must put ourselves in `development` mode:

    export FLASK_ENV=development
    flask run

Or, in Windows CMD:

    set FLASK_ENV=development
    flask run

## 1. Getting an arg

We have access to an object called `request`, which contains the data passed to the server when the request was made. For example, the headers passed by the browser (try printing `request.headers`), and query parameters (if any). 

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def root():
    """
    Get one argument as a string.
    """
    name = request.args.get('name', 'world')
    return f"Hello {name}!"

## 2. Calculator

In [ ]:
@app.route('/')
def root():
    """
    (2) A simple calculator with GET requests.
    """
    vp = float(request.args.get('vp') or 0)
    rho = float(request.args.get('rho') or 0)
    return f"Impedance: {vp * rho}"

<html><div style="background-color:#eeffee; border-radius: 8px; border: solid 1px #88aa88; padding: 0.5em 1em 1em 1em;">
            
<h3>EXERCISE</h3>

<p>Add another endpoint to the site to compute <a href="https://www.subsurfwiki.org/wiki/Gardner%27s_equation">Gardner's equation</a>, $\rho = 310 V_\mathrm{P}^{0.25}$ (or your favourite equation!).
    
You will need to add a function to handle GET requests on a new path.</p>
    
</div></html>

### Path parameters

As well as passing a query string like `?vp=2400&rho=2500`, which turns into a dictionary `{'vp': '2400', 'rho': '2500'}` (called `request.args`), we can also pass variables in the path itself.

 This is good for querying databases: path parameters represent entities (tables in your DB, more or less).
 
 Here's an example:

In [ ]:
@app.route('/hello/<name>')
def hello(name):
    return f"Hello {name}"

Here's [one explanation](https://stackoverflow.com/a/31261026/3381305) of why you might do this:

> Best practice for RESTful API design is that path params are used to identify a specific resource or resources, while query parameters are used to sort/filter those resources.

> Here's an example. Suppose you are implementing RESTful API endpoints for an entity called Car. You would structure your endpoints like this:

>     GET /cars
>     GET /cars/:id
>     POST /cars
>     PUT /cars/:id
>     DELETE /cars/:id

> This way you are only using path parameters when you are specifying which resource to fetch, but this does not sort/filter the resources in any way.

## 3. Classify image via URL given as query

To do this, we need some new functions:

- one to process the image as in training.
- one to fetch an image from a URL.
- one to make a prediction from an image.

First, here's the image processing function:

In [ ]:
def img_to_arr(img):
    """
    Apply the same processing we used in training: greyscale and resize.
    """
    img = img.convert(mode='L').resize((32, 32))
    return np.asarray(img).ravel() / 255

Here's the `fetch_image()` and `predict_from_image()` functions:

In [ ]:
def fetch_image(url):
    """
    Download an image from the web and pass to the image processing function.
    """
    r = requests.get(url)
    f = BytesIO(r.content)
    return Image.open(f) 


def predict_from_image(clf, img):
    """
    Classify an image.
    """
    arr = img_to_arr(img)
    X = np.atleast_2d(arr)
    probs = clf.predict_proba(X)
    result = {
        'class': clf.classes_[np.argmax(probs)],
        'prob': probs.max(),
        'classes': clf.classes_.tolist(),
        'probs': np.squeeze(probs).tolist(), # Must be serializable.
    }
    return result

Now we can write the prediction function for the app:

In [ ]:
import joblib
from flask import Flask, request, jsonify

app = Flask(__name__)

CLF = joblib.load('../app/data/rf.gz')

@app.route('/predict')
def predict():
    """
    (3) Make a prediction from a URL given via GET request.
    
    Using a URL means we can still just accept a string as an arg.

    There's still no human interface.
    """
    url = request.args.get('url')
    img = utils.fetch_image(url)
    result = utils.predict_from_image(CLF, img)

    # Deal with not getting a URL.
    # if url:
    #     img = utils.fetch_image(url)
    #     result = utils.predict_from_image(CLF, img)
    # else:
    #     result = 'Please provide a URL'

    return jsonify(result)

## 4. Provide URL via a GET form

First we'll just show a 'Hello world' page. The `simple_page.html` file can contain any HTML:

In [ ]:
from flask import render_template

@app.route('/simple')
def simple():
    """
    (4a) Render a template.
    """
    return render_template('simple_page.html')

It's tempting to defer all design etc until later, but I strongly recommend using some styling from the start so you can evolve it alongside the content. There are a couple of 'off the shelf' styles I use:

- [Bootstrap](https://getbootstrap.com/), the gold standard, originally from Twitter. Downside: looks like every other website in the universe.
- [new.css](https://newcss.net/), arguably the simplest possible HTML framework and the one I'm using here.

There are loads of others, eg [Skeleton](http://getskeleton.com/) and [Foundation](https://get.foundation/). Find more by Googling something like _responsive HTML5 frameworks_.

Now we can add a form:

In [ ]:
@app.route('/form', methods=['GET'])
def form():
    """
    (4b) Make a prediction from a URL given via a GET form.
    """
    url = request.args.get('url')
    if url:
        img = utils.fetch_image(url)
        result = utils.predict_from_image(CLF, img)
        result['url'] = url  # If we add this back, we can display it.
    else:
        result = {}

    return render_template('form.html', result=result)

The key part of the form HTML document, `form.html`, looks like this:

    <form action="/form" method="get">
        <input type="url" name="url" placeholder="Paste a URL" required="required" size=64 />
        <button type="submit">Predict</button>
    </form>
    
Then we have a bit of Jinja2 'code' in the page to show items from a `result` dictionary, if that dictionary contains any items:

    {% if result %}

        <p>{{ result['class'] }}: {{ "%.3f"|format(result['prob']) }}</p>

    {% endif %}
    
That's it!

<html><div style="background-color:#eeffee; border-radius: 8px; border: solid 1px #88aa88; padding: 0.5em 1em 1em 1em;">
            
<h3>EXERCISE</h3>

<p>Add an <strong>About</strong> page to the site. You will need to add a template (an HTML file), and a function to handle GET requests on the <code>/about</code> path.</p>
    
</div></html>

## GET vs POST

In a nutshell, `GET` is awesome because the data is passed in the URL. This means we can share the result of a query (say) just by sharing the URL with someone. But the URL is not secure, e.g. it's recorded in the server's logs, and it limits both the size and type of data we can pass to the server.

If we want to pass a lot of data, or structured data (e.g. a nested dictionary), or a bunch of non-text bytes, or we want to pass secure data, we'll need to use the `POST` method instead.

Our handlers may stil want to handle `GET` requests, since that's how a browser is going to ask for a resource.

We could convert the GET form to a post form simply by changing the `method` (and handling that on the back-end):

    <form action="/form" method="post">
        <input type="url" name="url" placeholder="Paste a URL" required="required" size=64 />
        <button type="submit">Predict</button>
    </form>

## 5. Upload image via a POST form

In [ ]:
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    """
    (5) Make a prediction from an image uploaded via a POST form.

    Bonus: on a mobile device, there will automatically be an option to
    capture via the camera.
    """
    if request.method == 'POST':
        data = request.files['image'].read()
        img = Image.open(BytesIO(data))
        result = utils.predict_from_image(CLF, img)
        # result['image'] = base64.b64encode(data).decode('utf-8')
    else:
        result = {}

    return render_template('upload.html', result=result)

The HTML for the form is like this:

    <form action="/upload" method="POST" enctype="multipart/form-data">
        <input type="file" accept="image/*" name="image" required="required" />
        <button type="submit" name="action">Upload</button>
    </form>

The `enctype` parameter is needed for sending more complex data types such as binary files. The default is `enctype="multipart/form-data"` ([more about this](https://www.w3schools.com/tags/att_form_enctype.asp)).

### Displaying the image

We can send the image back to the front-end by adding a base64-encoded string to the `result` dictionary (uncomment the line in the handler function above). Then we can display the image with some HTML:

    <img src="data:image/png;base64,{{ result['image'] }}" />
    
`base64` is an encoding similar to binary, octal and hexadecimal, but with 64 symbols instead of 2, 8 and 16 respectively. Each symbol therefore represents 6 bits of data (2<sup>6</sup> = 64), so you can pack 3 bytes into 4 symbols. [Read more about base64 encoding.](https://en.wikipedia.org/wiki/Base64)

## 6. Sending a plot back to the user

In a simple app, I usually try to reduce complexity as much as possible. So it's useful to know how to store a `matplotlib` plot in memory, by tricking it into thinking we're saving a file, then sending those bytes to the front-end.

Here's the code to save the `matplotlib` plot:

In [ ]:
import matplotlib.pyplot as plt
from io import BytesIO
import base64

fig, ax = plt.subplots()
ax.plot([0, 1, 2, 3, 4, 5])

# Put in memory.
handle = BytesIO()
plt.savefig(handle, format='png', facecolor=fig.get_facecolor())
plt.close()

# Encode.
handle.seek(0)
image_string = base64.b64encode(handle.getvalue()).decode('utf8')

Now we add one line to the handler:

In [ ]:
result['plot'] = utils.plot(result['probs'], CLF.classes_)

## 7. A web API for POST requests

Instead of passing data via a form, programmers would often like to pass data to a server programmatically. To handle requests like this, we can write a special endpoint.

We'll always want to return JSON from an endpoint like this, so we don't need templates or any HTML:

In [ ]:
@app.route('/post', methods=['POST'])
def post():
    """
    (7) Make a prediction from a URL provided via POST request.
    """
    url = request.json.get('url')
    img = utils.fetch_image(url)
    result = utils.predict_from_image(CLF, img)
    return jsonify(result)

## 8. A web API handling images or URLs

If accessing the web API from code, you may not have a URL to pass to the service, and there is no form for doing a file upload. So we need a way to pass the image as data. There are lots of ways to do this; one way is to encode as base64.

In [ ]:
@app.route('/api/v0.1', methods=['POST'])
def api():
    """
    (8) Extend example (7) to make a prediction from a URL or a
    base64-encoded image via POST request. 
    """
    data = request.json.get('image')
    if data.startswith('http'):
        img = utils.fetch_image(url)
    else:
        img = Image.open(BytesIO(base64.b64decode(data)))
    result = utils.predict_from_image(CLF, img)
    return jsonify(result)

Check out [Hitting_our_web_API.ipynb](./Hitting_our_web_API.ipynb) to see how to use this API.